## **Catégorisez automatiquement des questions**

### partie 4/8 : Prédiction de tags, approche supervisée + tracking mlflow

#### <br> Notebook d’exploration et de pré-traitement des questions, comprenant une analyse univariée et multivariée, un nettoyage des questions, un feature engineering de type bag of words avec réduction de dimension (du vocabulaire et des tags) 

<br>


## 1 Import


### 1.1 Librairies


In [1]:
# finir d'organiser, check inutiles ?

#
import os, sys, random
import ast
import numpy as np
import pandas as pd
from typing import Tuple
from pandarallel import pandarallel
from pprint import pprint
import json
import requests
from collections import Counter

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image

# NLP
from bs4 import BeautifulSoup
import re, string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

#
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import corpus2dense
from gensim.matutils import Sparse2Corpus
from gensim.models import LdaModel, CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim import similarities
from gensim.models.ldamulticore import LdaMulticore

#
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import make_scorer, PredictionErrorDisplay, r2_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor


print('\nPython version ' + sys.version)
print('pyLDAvis version ' + pyLDAvis.__version__)

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)

#
import pickle
import mlflow
from mlflow import MlflowClient
from mlflow.models.signature import infer_signature, ModelSignature #, Schema, ParamSchema
from mlflow.types import Schema, ParamSchema, ParamSpec, ColSpec

# os.environ['MLFLOW_TRACKING_URI'] = './'

# ! REQUIRES CONSOLE COMMAND : mlflow ui
# depuis dossier notebooks

# Utilisable seulement en local...
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")


[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!



Python version 3.11.5 (main, Sep 11 2023, 13:23:44) [GCC 11.2.0]
pyLDAvis version 3.4.0

Number of CPU cores: 8
INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### 1.2 Functions


In [2]:
def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))


def preprocess_text(text):
    #Cleaning
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = text.lower().strip()

    # Tokenization
    try:
        tokens = nltk.word_tokenize(text)
        tokenizer = nltk.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(" ".join(tokens))  # Apply RegexpTokenizer to the entire list

        # Remove punctuation (make sure, RegexpTokenizer should have done it already)
        tokens = [token for token in tokens if token not in string.punctuation]

    except Exception as e:
        print(f"Error in tokenization: {e}")
        return []

    # Lemmatization
    lemmatizer = WordNetLemmatizer()

    # Get part of speech for each token
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = []

    for token, pos_tag in pos_tags:
        # ! Uncommenting next line may crash the cell
        # print(f"Token: {token}, POS Tag: {pos_tag}")
        if pos_tag.startswith('V'):
            # On garde
            lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='v'))
            # Returns the input word unchanged if it cannot be found in WordNet.
        elif pos_tag.startswith('N'):
            # On garde
            try:
                lemmatized_tokens.append(lemmatizer.lemmatize(token, pos='n'))
            except Exception as e:
                print(f"Error lemmatizing verb {token}: {e}")
        # Sinon on supprime

    # Read forbidden words (stopwords, too frequent, too rare) from the file
    with open('./../0_data/cleaned_data/forbidden_words.txt', 'r') as file:
        forbidden = [line.strip() for line in file]

    filtered_list = [token for token in lemmatized_tokens if token not in forbidden]

    # keep uniques
    seen_tokens = set()
    unique_tokens = []

    for token in filtered_list:
        if token not in seen_tokens:
            seen_tokens.add(token)
            if len(token) > 2:
                unique_tokens.append(token)

    return unique_tokens


In [3]:
def create_mlflow_experiment(
    experiment_name: str, artifact_location: str, tags: dict[str, str]
) -> str:
    """
    Create a new mlflow experiment with the given name and artifact location.

    Parameters:
    ----------
    experiment_name: str
        The name of the experiment to create.
    artifact_location: str
        The artifact location of the experiment to create.
    tags: dict[str,Any]
        The tags of the experiment to create.

    Returns:
    -------
    experiment_id: str
        The id of the created experiment.
    """
    try:
        experiment_id = mlflow.create_experiment(
            name=experiment_name, artifact_location=artifact_location, tags=tags
        )
    except:
        print(f"Experiment {experiment_name} already exists.")
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    mlflow.set_experiment(experiment_name=experiment_name)

    return experiment_id


def get_mlflow_experiment(
    experiment_id: str = None, experiment_name: str = None
) -> mlflow.entities.Experiment:
    """
    Retrieve the mlflow experiment with the given id or name.

    Parameters:
    ----------
    experiment_id: str
        The id of the experiment to retrieve.
    experiment_name: str
        The name of the experiment to retrieve.

    Returns:
    -------
    experiment: mlflow.entities.Experiment
        The mlflow experiment with the given id or name.
    """
    if experiment_id is not None:
        experiment = mlflow.get_experiment(experiment_id)
    elif experiment_name is not None:
        experiment = mlflow.get_experiment_by_name(experiment_name)
    else:
        raise ValueError("Either experiment_id or experiment_name must be provided.")

    return experiment


def turn_str_back_into_list(df):
    """Correct the type change due to .csv export"""

    df['title_nltk'] = df['title_nltk'].apply(ast.literal_eval)
    df['body_nltk'] = df['body_nltk'].apply(ast.literal_eval)
    df['title_spacy'] = df['title_spacy'].apply(ast.literal_eval)
    df['body_spacy'] = df['body_spacy'].apply(ast.literal_eval)
    df['all_tags'] = df['all_tags'].apply(ast.literal_eval)



In [4]:
# Again, this needs mlfow ui console command first -> unusable on remote server
# all_experiments = client.search_experiments()
# pprint(all_experiments)


### 1.3 Data


In [5]:
train = pd.read_csv('./../0_data/cleaned_data/train_bow_uniques.csv', sep=',')
test = pd.read_csv('./../0_data/cleaned_data/test_bow_uniques.csv', sep=',')

turn_str_back_into_list(train)
turn_str_back_into_list(test)

display(train.tail())

train.shape
print(test.shape)


,CreationDate,title,body,all_tags,title_nltk,body_nltk,title_spacy,body_spacy
42893,2017-02-23 11:34:31,Do we need clear MDC after HTTP request in Spring,According to this answer thread local variable...,"[java, spring, logging, log4j, logback]","[need, mdc, request, spring]","[need, mdc, request, spring, accord, answer, t...","[need, request]","[accord, answer, thread, variable, use, clear,..."
42894,2011-10-13 20:57:32,How to make i18n with Handlebars.js (mustache ...,I'm currently using Handlebars.js (associated ...,"[javascript, jquery, internationalization, han...","[make, i18n, handlebar, template]","[make, i18n, handlebar, template, associate, b...",[template],"[associate, web, app, client, render, issue, w..."
42895,2012-09-06 00:16:46,How can I make R read my environmental variables?,I am running R on EC2 spot instances and I nee...,"[linux, r, ubuntu, amazon-ec2, environment-var...","[make, read, variable]","[make, read, variable, run, spot, instance, ne...","[read, variable]","[run, spot, instance, need, terminate, cancel,..."
42896,2021-03-23 03:50:50,How to prevent react-query from fetching initi...,I'm using react-query v3.13 to fetch data from...,"[javascript, reactjs, fetch, react-query, swr]","[prevent, query, fetch, enable]","[prevent, query, fetch, enable, data, want, po...","[prevent, react, query, fetch, enable]","[react, query, fetch, datum, want, api, point,..."
42897,2016-03-17 04:19:15,Inserting into table with an Identity column w...,I have a table A_tbl in my database. I have cr...,"[sql, sql-server, database, ssms, database-rep...","[insert, table, identity, column, replication,...","[insert, table, identity, column, replication,...","[insert, table, column, replication, cause, er...","[table, database, create, trigger, capture, in..."


(4767, 8)


## 2 KNN


In [6]:
# Often gives good results if enough data
# fast
# Accepts basically any input, as long as it is numerical

# => Perfect for testing different embeddings !


### 2.1 Dummy knn : il copie sur le + proche voisin


In [7]:
# Notre baseline

def predict_tags_using_dummy_knn(df, feature, target, k=1, exemple=None):
    documents = df[feature].tolist()
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Convert Gensim corpus to dense matrix
    dense_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T

    # Ici on ne va pas demander au knn de faire de prediction,
    # On veut juste qu'il trouve les voisins.
    # Mais la fonction fit a besoin de targets en param
    target_values = df[target].values

    # Initialize kNN model
    knn_model = KNeighborsRegressor(n_neighbors=k, metric='cosine', algorithm='brute')
    # print(knn_model.n_neighbors)

    knn_model.fit(dense_matrix, target_values)

    # Example query
    query_document = exemple
    query_bow = gensim_dictionary.doc2bow(query_document)
    query_vector = corpus2dense([query_bow], num_terms=len(gensim_dictionary)).T

    # Find nearest neighbors
    _, indices = knn_model.kneighbors(query_vector)

    # Aggregate tags from neighbors
    neighbor_tags = [tag for i in indices.flatten() for tag in df.iloc[i][target]]

    print(neighbor_tags)

    # Predict tags based on most common tags among neighbors
    predicted_tags = [tag for tag, _ in Counter(neighbor_tags).most_common(n=10)]
    # 5 tags/question en moyenne mais on peut suggérer +
    # ici a ameliorer

    return predicted_tags, knn_model


exemple = ["your", 'text', 'document', 'javascript']
# Call the function with your DataFrame and the desired text feature and target tags
predicted_tags, knn_test = predict_tags_using_dummy_knn(train, 'title_nltk', 'all_tags', exemple=exemple)
print(predicted_tags, '\n')

# javascript ok


['javascript', 'internet-explorer', 'class', 'internet-explorer-8', 'classname']
['javascript', 'internet-explorer', 'class', 'internet-explorer-8', 'classname'] 



### 2.2 knn basic


In [8]:
# add grid search cv
# add score

def predict_tags_using_knn(df, feature, target, k=50, exemple=None):
    documents = df[feature].tolist()
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Convert Gensim corpus to dense matrix
    dense_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T

    # Ici on ne va pas demander au knn de faire de prediction,
    # On veut juste qu'il trouve les voisins.
    # Mais la fonction fit a besoin de targets en param
    target_values = df[target].values

    # Initialize kNN model
    knn_model = KNeighborsRegressor(n_neighbors=k, metric='cosine', algorithm='brute')
    knn_model.fit(dense_matrix, target_values)

    # Example query
    query_document = exemple
    query_bow = gensim_dictionary.doc2bow(query_document)
    query_vector = corpus2dense([query_bow], num_terms=len(gensim_dictionary)).T

    # Find nearest neighbors
    _, indices = knn_model.kneighbors(query_vector)

    # Aggregate tags from neighbors
    neighbor_tags = [tag for i in indices.flatten() for tag in df.iloc[i][target]]

    print(neighbor_tags)

    # Predict tags based on most common tags among neighbors
    predicted_tags = [tag for tag, _ in Counter(neighbor_tags).most_common(n=10)]
    # 5 tags/question en moyenne mais on peut suggérer +
    # ici a ameliorer

    return query_vector, predicted_tags


### 2.3 predictions (input = list)


In [14]:
exemple1 = ["your", 'text', 'document', 'javascript']
_, predicted_tags1 = predict_tags_using_knn(train, 'title_nltk', 'all_tags', exemple=exemple1)
print(predicted_tags1, '\n')

# javascript ok


['c#', '.net', 'ms-word', 'openxml', 'openxml-sdk', 'javascript', 'internet-explorer', 'class', 'internet-explorer-8', 'classname', 'java', 'javascript', 'ajax', 'selenium', 'htmlunit-driver', 'c#', '.net', 'windows', 'f#', 'console', 'javascript', 'jquery', 'jquery-plugins', 'text-to-speech', 'html5-audio', 'javascript', 'html', 'css', 'text', 'truncate', 'javascript', 'jquery', 'css', 'dom', 'document', 'javascript', 'html', 'string', 'text', 'extract', 'javascript', 'dom', 'substring', 'indexof', 'getselection', 'javascript', 'html', 'function', 'text', 'onclick', 'c#', '.net', 'html', 'pdf', 'extract', 'javascript', 'jquery', 'css', 'copy', 'cut', 'javascript', 'html', 'url', 'base64', 'data-uri', 'python', 'module', 'preprocessor', 'nlp', 'stemming', 'javascript', 'php', 'jquery', 'curl', 'http-headers', 'python', 'text', 'replace', 'ms-word', 'python-docx', 'c#', 'javascript', 'html', 'http', 'dom', 'javascript', 'jquery', 'ruby-on-rails', 'tdd', 'jasmine', 'ios', 'swift', 'heigh

In [15]:
exemple2 = ["your", 'text', 'document', 'python']
# Call the function with your DataFrame and the desired text feature and target tags
_, predicted_tags2 = predict_tags_using_knn(train, 'title_nltk', 'all_tags', exemple=exemple2)
print(predicted_tags2, '\n')

# python


['python', 'module', 'preprocessor', 'nlp', 'stemming', 'c#', '.net', 'ms-word', 'openxml', 'openxml-sdk', 'python', 'parsing', 'text', 'file-io', 'python-2.7', 'python-2.7', 'ubuntu', 'python-3.x', 'spatial-index', 'r-tree', 'python', 'plot', 'tree', 'data-visualization', 'visualization', 'c#', '.net', 'windows', 'f#', 'console', 'python', 'html', 'web-scraping', 'text', 'beautifulsoup', 'python', 'python-3.x', 'algorithm', 'sorting', 'mergesort', 'python', 'python-2.7', 'reflection', 'delegation', 'message-passing', 'python', 'python-3.x', 'annotations', 'lint', 'type-hinting', 'python', 'pdf', 'python-3.7', 'pypdf', 'pdf-extraction', 'python', 'selenium', 'selenium-webdriver', 'xpath', 'webdriverwait', 'python', 'macos', 'python-3.x', 'sublimetext2', 'sublimetext', 'python-3.x', 'pdf', 'text', 'extract', 'pdfminer', 'python', 'documentation', 'python-3.7', 'docstring', 'python-dataclasses', 'python', 'text', 'stemming', 'plural', 'singular', 'nlp', 'cluster-analysis', 'data-mining',

In [16]:
exemple = test['title_nltk'][0]
print(exemple)

# Call the function with your DataFrame and the desired text feature and target tags
_, predicted_tags = predict_tags_using_knn(train, 'title_nltk', 'all_tags', exemple=exemple)
print(predicted_tags, '\n')

# firebase peut etre predit
# grand succes !


['find', 'class', 'com', 'google', 'firebase', 'provider']
['java', 'spring', 'rest', 'gradle', 'spring-boot', 'java', 'android', 'firebase', 'gradle', 'android-gradle-plugin', 'android', 'android-studio', 'firebase', 'android-gradle-plugin', 'google-play-services', 'android', 'android-intent', 'arraylist', 'unmarshalling', 'parcelable', 'php', 'class', 'laravel', 'alias', 'autoloader', 'php', 'sql', 'laravel', 'laravel-5', 'laravel-artisan', 'java', 'spring', 'spring-boot', 'spring-security', 'spring-security-oauth2', 'java', 'maven', 'maven-2', 'maven-3', 'protocol-buffers', 'android', 'google-maps', 'dictionary', 'android-mapview', 'inflate', 'android', 'android-studio', 'flutter', 'sdk', 'android-sdk-manager', 'java', 'spring', 'junit', 'spring-boot', 'spring-data', 'php', 'laravel', 'https', 'laravel-valet', 'valet', 'spring', 'maven', 'spring-mvc', 'spring-boot', 'spring-profiles', 'json', 'angular', 'typescript', 'jwt', 'guard', 'c#', '.net', 'visual-studio-2012', 'compression',

### 2.4 Predictions (input = text)


In [17]:
exemple = test['title'][0]
print(exemple)

exemple_text = preprocess_text(exemple)

_, predicted_tags = predict_tags_using_knn(train, 'title_nltk', 'all_tags', exemple=exemple_text)
print(predicted_tags, '\n')

# ok


Didn't find class "com.google.firebase.provider.FirebaseInitProvider"
['java', 'spring', 'rest', 'gradle', 'spring-boot', 'java', 'android', 'firebase', 'gradle', 'android-gradle-plugin', 'android', 'android-studio', 'firebase', 'android-gradle-plugin', 'google-play-services', 'android', 'android-intent', 'arraylist', 'unmarshalling', 'parcelable', 'php', 'class', 'laravel', 'alias', 'autoloader', 'php', 'sql', 'laravel', 'laravel-5', 'laravel-artisan', 'java', 'spring', 'spring-boot', 'spring-security', 'spring-security-oauth2', 'java', 'maven', 'maven-2', 'maven-3', 'protocol-buffers', 'android', 'google-maps', 'dictionary', 'android-mapview', 'inflate', 'android', 'android-studio', 'flutter', 'sdk', 'android-sdk-manager', 'java', 'spring', 'junit', 'spring-boot', 'spring-data', 'php', 'laravel', 'https', 'laravel-valet', 'valet', 'spring', 'maven', 'spring-mvc', 'spring-boot', 'spring-profiles', 'json', 'angular', 'typescript', 'jwt', 'guard', 'c#', '.net', 'visual-studio-2012', 'co

In [ ]:
exemple = test['title'][1]
print(exemple)

exemple_text = preprocess_text(exemple)

_, predicted_tags = predict_tags_using_knn(train, 'title_nltk', 'all_tags', exemple=exemple)
print(predicted_tags, '\n')

# scala ok


### 2.5 scores


In [14]:
# Notre objectif de prédiction de tags ressemble a un pb de classification multi-label,
# où la matrice de confusion est extrêmement déséquilibrée :
# 5 tags sont prédits positifs, contre environ 250 000 tags (si on travaille sur all_tags)
# predits negatifs. Autrement dit :

# On peut utiliser la precision pour évaluer notre modèle. C'est même exactement l'outil qu'il nous faut :
# "précision = la proportion de prédictions correctes parmi les points que l’on a prédits positifs."
# En + c de loin le plus léger en ressources, puisqu'il ne s'occupe que des 5 tags prédits.

# En revanche je pense que le recall n'a pas vraiment de sens ici, il sera "écrasé" par
# le nombre de tags predits negatifs, sa valeur sera tjs très proche de zero.
# (même remarque pour la spécificité et l'accuracy)
# Et sans recall, pas de f1 score.
# à vérifier


#### Precision


In [15]:
# pourquoi on ne peut pas utiliser le score precision sckikit ici :
from sklearn.metrics import precision_score as p_score

# Assuming y_true is the ground truth (real tags) and y_pred is the predicted tags
precision = p_score(['ok', 'ko'], ['ko', 'ok'], average='micro')  # You can use 'micro', 'macro', or 'weighted' depending on your use case
print(f'Precision: {precision}')


Precision: 0.0


In [16]:
def precision_topics(real_tags:list, predicted_tags:list): # pour comparer 2 listes
    # precision = TP / (TP + FP)
    tp = 0
    for predicted_tag in predicted_tags:
        if predicted_tag in real_tags:
            tp += 1

    fp = len(predicted_tags) - tp
    precision = tp/(tp + fp)
    # <=> precision = tp/len(predicted_tags)

    return precision


print(precision_topics(exemple1, predicted_tags1))
precision_topics(exemple2, predicted_tags2)

# ok


0.0


/tmp/ipykernel_130447/636444049.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if predicted_tag in real_tags:


0.0

In [17]:
def precision_score(y_true, y_pred): # pour comparer 2 df ou 2 matrices de mm shape[0]
    precision = 0
    for i in range(0, len(y_pred)):
        precision += precision_topics(y_true[i], y_pred[i])
    precision_moyenne = precision / len(y_pred)

    return precision_moyenne


# pour la gridsearchcv
custom_precision_scorer = make_scorer(precision_score, greater_is_better=True)


#### Recall


In [18]:
# La precision nous interesse davantage ici, pour au moins 2 raisons :
# 1) On n'a aucune raison de "pénaliser" le modele pour les faux negatifs
# 2) Ici le recall va etre tres proche de zero, et sa variance sera tres faible
# => bcp - parlant

def recall_topics(all_tags: list, predicted_tags: list):
    # recall = TP / (TP + FN)
    tp = 0
    fn = 0
    for real_tag in all_tags:
        if real_tag in predicted_tags:
            tp += 1
        else:
            fn += 1
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    return recall


# insister ds la doctype : y_all = all tags ici, != y_true (5-6 tags max)
def recall_score(y_all, y_pred): # pour comparer 2 df ou 2 matrices de mm shape[0]
    recall = 0
    for i in range(0, len(y_pred)):
        recall += recall_topics(y_all, y_pred[i]) # ca risque d'etre long a calculer
    recall_moyen = recall / len(y_pred)

    return recall_moyen


custom_recall_scorer = make_scorer(recall_score, greater_is_better=True)


#### f1 score


In [19]:
# mm rearques que pour le recall

def f1_topics(real_tags: list, predicted_tags: list, all_tags:list):
    precision = precision_score(real_tags, predicted_tags)
    recall = recall_topics(all_tags, predicted_tags)

    # F1 score = 2 * (precision * recall) / (precision + recall)
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1


def f1_score(y_true, y_pred, y_all):
    f1_score = 0
    for i in range(0, len(y_pred)):
        score += f1_topics(y_true[i], y_pred[i], y_all)
    score_moyen = score / len(y_pred)

    return score_moyen


custom_f1_scorer = make_scorer(f1_score, greater_is_better=True)


#### Accuracy


In [20]:
# idem, ici la variance sera presque nulle (TP entre 0 et 5, TN = environ 250 000...)

def accuracy_topics(real_tags: list, predicted_tags: list, all_tags:list):
    # accuracy = (TP + TN) / (TP + TN + FP + FN)
    tp = sum(1 for tag in predicted_tags if tag in real_tags)
    tn = sum(1 for tag in all_tags if (tag not in predicted_tags) and (tag not in real_tags) )
    fp = sum(1 for tag in predicted_tags if tag not in real_tags)
    fn = sum(1 for tag in all_tags if (tag not in predicted_tags) and (tag in real_tags))

    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0

    return accuracy


def accuracy_score(y_true, y_pred, y_all):
    score = 0
    for i in range(0, len(y_pred)):
        score += accuracy_topics(y_true[i], y_pred[i], y_all)
    score_moyen = score / len(y_pred)

    return score_moyen


custom_accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)


#### Jaccard similarity


In [21]:
# utilisé en partie 3 pour évaluer la similarité des topics obtenus / la lda
# ici pour comparer topics reels et topics predits.
# devrait etre correlé a la precision non ?
# st ts les 2 liés au nb de tp (true positifs)

def jaccard_similarity(topic1, topic2):
    set1 = set(topic1)
    set2 = set(topic2)
    return len(set1.intersection(set2)) / len(set1.union(set2))


def jaccard_score(y_true, y_pred): # pour comparer 2 df ou 2 matrices de mm shape[0]
    jacc = 0
    for i in range(0, len(y_pred)):
        jacc += jaccard_similarity(y_true[i], y_pred[i])
    jacc_moyen = jacc / len(y_pred)

    return jacc_moyen


# pour la gridsearchcv
custom_jacc_scorer = make_scorer(jaccard_score, greater_is_better=True)


### 2.6 Evaluation


In [22]:
# add grid search cv
# add recall, f1 score ?

def predict_tags_using_knn(train_df=train, feature='body_nltk', target='all_tags', test_df=test, k=5, n=5,
                           scorer=precision_score):
    # 1 PREPROCESSING
    documents = train_df[feature].tolist()
    gensim_dictionary = Dictionary(documents)
    print(len(gensim_dictionary))
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]
    print(len(corpus))
    # taille corpus ?

    # Convert Gensim corpus to dense matrix
    dense_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T
    # taille matrice ? afficher
    print(dense_matrix.shape, '\n')
    # pas tres dense ici, c notre bow donc tres sparse en fait
    # curieux d'appeler "corpus2dense()" une fonction qui retourne une matrice sparse
    pprint(dense_matrix[:10]) # vraiment tres dense, quasiment que des 0 ! Bref
    print('\n')

    # Ici on ne va pas demander au knn de faire de prediction,
    # On veut juste qu'il trouve les voisins.
    # Mais la fonction fit a besoin de targets en param
    target_values = train_df[target].values

    # 2 MODEL TRAINING
    knn_model = KNeighborsRegressor(n_neighbors=k)
    knn_model.fit(dense_matrix, target_values)

    # 3 PREDICTION
    # Predictions completes en 1h ou 2
    # optimiser avec pandarallel ?
    # use a sample en attendant
    predictions=[]
    min_range=2500
    max_range=2511 # test.shape[0]=4767
    for i in range(min_range, max_range):
        query_document = test_df[feature][i]
        print(f'doc {i} : {query_document}')
        print(f'real tags : {test[target][i]}')
        query_bow = gensim_dictionary.doc2bow(query_document)
        query_vector = corpus2dense([query_bow], num_terms=len(gensim_dictionary)).T

        # Find nearest neighbors
        _, indices = knn_model.kneighbors(query_vector)

        # Aggregate tags from neighbors
        neighbor_tags = [tag for i in indices.flatten() for tag in train_df.iloc[i][target]]

        # Predict tags based on most common tags among neighbors
        predicted_tags = [tag for tag, _ in Counter(neighbor_tags).most_common(n=n)]
        # 5 tags/question en moyenne mais on peut suggérer +
        predictions.append(predicted_tags)
        print(f'predicted : {predicted_tags}')
        print(precision_topics(test_df[target][i], predicted_tags), '\n')

    true_tags = [tags for tags in test_df[target][min_range:max_range]]

    # mean_precision = precision_score(true_tags, predictions)
    mean_precision = scorer(true_tags, predictions)

    print(f'precision moyenne = {mean_precision}')

    return mean_precision


# Call the function with your DataFrame and the desired text feature and target tags
predict_tags_using_knn(scorer=jaccard_score)

# 0.34 de precision ?? c enorme !


8513
42898
(42898, 8513) 

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)


doc 2500 : ['let', 'support', 'element', 'try', 'get', 'application', 'run', 'stop', 'template', 'error', 'thead', 'tbody', 'row', 'index', 'datepicker', 'sec', 'html', 'syntaxerror', 'compiler', 'parse', 'set', 'foreach', 'figure', 'need', 'follow', 'topic', 'issue', 'state', 'bootstrap', 'beta', 'doesnt', 'work', 'http', 'github', 'com', 'software', 'help', 'appreciate']
real tags : ['html', 'angular', 'npm', 'frontend', 'ngx-bootstrap']
predicted : ['c#', '.net', 'null', 'int', 'nullable']
0.0 

doc 2501 : ['java', 'string', 'modify', 'search', 'representation', 'get', 'material', 'look', 'com', 'cpp', 'misc', 'multi', 'article', 'php', 'say', 'text', 'support', 'modification', 'serialization', 'wikiped

0.10606060606060606

### 2.7 Recherche manuelle du nb optimal de voisins


In [23]:
# Prend du temps
# décommenter pour tester

precision = []
for k in range(45, 50): # pour un classifieur binaire il faut k impair, mais ici comme on veut
    print(f'k={k}', '\n')
    # precision.append((k, predict_tags_using_knn(k=k)))

    print('\n')


k=45 



k=46 



k=47 



k=48 



k=49 





In [24]:
# Nécessite cellule précédente décommentée

# body
# 0.33 pour k=5
# 0.37 pour k proche de 50 !!

record_max = 0
for k, result in precision:
    print(f'For {k} neighbors : precision moyenne = {result}')
    if result > record_max:
        record_max = result

print('\n', f' meilleur score = {record_max}')

# sur 'title_nltk' :
# best : 0.29 pour k=47-48
# = 1 quart des tags predits correctement
# pas mal !

# mieux sur body que sur title (heureusement !)

# tester predire seulement 3 ou 4 tags ?



  meilleur score = 0


### 2.8 Influence du nb de topics prédits sur la précision


In [25]:
# idem, takes time
# On fera une véritable recherche d'optimisation + tard avec MLFlow

precision4 = []
precision3 = []
for k in range(50, 151, 10): # pour un classifieur binaire il faut k impair, mais ici comme on veut
    # encore que ?? les nb impairs semblent obtenir de meilleurs resultats
    print(f'k={k}', '\n')
    # precision4.append((k, predict_tags_using_knn(k=k, n=4)))
    # precision3.append((k, predict_tags_using_knn(k=k, n=3)))

    print('\n')


k=50 



k=60 



k=70 



k=80 



k=90 



k=100 



k=110 



k=120 



k=130 



k=140 



k=150 





In [26]:
# pas touche !
# > 0.5 !!

# factoriser ?
score_max4 = 0
for k, result in precision4:
    print(f'For {k} neighbors : precision moyenne = {result}')
    if result > score_max4:
        score_max4 = result

score_max3 = 0
for k, result in precision3:
    print(f'For {k} neighbors : precision moyenne = {result}')
    if result > score_max3:
        score_max3 = result

print('\n', f' meilleur score (4 tags predits) = {score_max4}')
print('\n', f' meilleur score (3 tags predits) = {score_max3}')

# 26

# tester k > 100 ?
# test en reduisant le corpus de tags ? ou pas (conserver la richesse du corpus ?)
# tester ce qui prend le + de tps (1 predict = environ 10 secondes, trop long)



  meilleur score (4 tags predits) = 0

  meilleur score (3 tags predits) = 0


In [27]:
# yes !

score_max4 = 0
for k, result in precision4:
    print(f'For {k} neighbors : precision moyenne = {result}')
    if result > score_max4:
        score_max4 = result


score_max3 = 0
for k, result in precision3:
    print(f'For {k} neighbors : precision moyenne = {result}')
    if result > score_max3:
        score_max3 = result


print('\n', f' meilleur score (4 tags predits) = {score_max4}')
print('\n', f' meilleur score (3 tags predits) = {score_max3}')

# near 60



  meilleur score (4 tags predits) = 0

  meilleur score (3 tags predits) = 0


### 2.9 Preprocessing (feature et target)


In [28]:
# Nous avons besoin de transformer la target pdt le preprocessing (-> bow)
# mm si on ne s'sn sert pas vraiment, car grid_search.fit() n'accepte que des valeurs numériques.

# Avantage : on peut utiliser des metriques classiques pour le score (ici r2),
# mais ca n'a aucun sens metier interpretable.
# peut tjs etre utile si fortement correlé à notre precision score custom
# ou au score de similarité Jaccard

# Compliqué à faire dans le pipeline sckikit, qui transforme les features mais pas la target.
# TransformedTargetRegressor ne convient pas non plus ici : c'est un modele wrapper,
# utilisé apres le pipeline.
# trouvé qq "solutions" + ou - elegantes, mais rien de compatible à la fois avec sklearn et mlflow.

# Ici convertir les tags en bag of words ou les one hot encoder revient exactement au meme, donc
# autant utiliser le bow, on a deja le transformeur.
# En + avec la plupart des embedding on peut considerer que la prediction de topics devient bien plus
# un probleme de regression que de classification, les valeurs predites par les modeles sont des vecteurs
# dans un espace techniquement fini, mais immense.


In [29]:
def token_list_into_bow(X):
    documents = X.tolist()
    # print(documents)
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Convert Gensim corpus to dense matrix
    bow_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T

    return bow_matrix


### 2.10 (tentative de) gridsearch


In [30]:
# Prend trop de ressources ! Il est tps d'utiliser les nested runs de mlflow.
# Parce qu'une gridsearch qui ne peut tester qu'une seule valeur pour un seul hyperparam a la fois,
# ce n'est pas tres utile...

# En plus je me demande si la gridsearch est une bonne idée pour le knn.
# deja pour la lda et la nmf j'avais des doutes
# a verifier mais pour ces modeles je pense qu'un dataset d'entrainement plus petit peut
# serieusement impacter le nb optimal pour l'hyperparam
# (voisins pour le knn, topics pour lda et nmf)

def pipe_knn(train_df=train, feature='title_nltk', target='all_tags', test_df=test, metric='cosine', graph=True):
    # Load your training data and labels
    X_train = train_df[feature].values
    y_train = train_df[target].values

    X_bow_matrix = token_list_into_bow(X_train)
    y_bow_matrix = token_list_into_bow(y_train)

    # Create a KNN Regressor
    knn_regressor = KNeighborsRegressor(metric=metric)

    # Create a pipeline with preprocessing and a knn regressor, to simplify gridsearch
    pipe = Pipeline(steps=[
        ("knn_regressor", knn_regressor)
    ])

    # Define hyperparameters and their possible values for grid search
    param_grid = {
        'knn_regressor__n_neighbors': [1],
        'knn_regressor__weights': ['uniform'] # , 'distance'
    }

    # Create the GridSearchCV object with multiple scoring metrics
    # scoring = {'neg_mean_squared_error': 'neg_mean_squared_error', 'r2': 'r2'}
    grid_search = GridSearchCV(pipe, param_grid=param_grid,
                            scoring='r2', cv=5, verbose=1) # add, refit='precision' for multiple scoring

    # Fit the GridSearchCV object to your training data to perform hyperparameter tuning
    grid_search.fit(X_bow_matrix, y_bow_matrix)

    # Access the best hyperparameters
    best_params = grid_search.best_params_
    print("Best Hyperparameters:", best_params)

    # Create the KNN regressor with the best hyperparameters
    best_knn_regressor = KNeighborsRegressor(# metric=metric,
                                             n_neighbors=best_params['knn_regressor__n_neighbors'],
                                             weights=best_params['knn_regressor__weights'])

    # Create a pipeline with the preprocessor and the tuned knn regressor
    pipeline_with_tuned_knn = Pipeline(steps=[
        ("knn_regressor", best_knn_regressor)  # Use the tuned neighbor and weight values here
    ])

    # Perform cross-validation (on training set) and display the scores for each split
    # scoring = ['r2', 'neg_mean_squared_error']
    cv_scores = cross_validate(pipeline_with_tuned_knn, X_bow_matrix, y_bow_matrix, cv=5, scoring='r2')
    # print("Cross-Validation Scores (training):", '\n', cv_scores)
    print("Cross-Validation Scores:")
    pprint(cv_scores)
    for i, score in enumerate(cv_scores['test_score']):
        print(f"Split {i+1} : precision = {score}")


# pipe_knn()


### 2.11 test mlflow run


In [31]:
# Si erreur, commande console
# mlflow ui
# depuis dossier notebooks

experiment_id = create_mlflow_experiment(
    experiment_name="knn_optimisation",
    artifact_location="./artifacts",
    tags={"modele": "knn", "feature": "title", 'nlp': 'nltk'},
)


Experiment knn_optimisation already exists.


MlflowException: API request to http://localhost:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=knn_optimisation (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f77b615ee90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="testing", experiment_id=experiment_id) as run:

    # log model using autolog
    # mlflow.autolog()
    mlflow.sklearn.autolog()
    pipe_knn()

    # print run info
    print("run_id: {}".format(run.info.run_id))
    print("experiment_id: {}".format(run.info.experiment_id))
    print("Artifact Location: {}".format(experiment.artifact_location))
    print("status: {}".format(run.info.status))
    print("start_time: {}".format(run.info.start_time))
    print("end_time: {}".format(run.info.end_time))
    print("lifecycle_stage: {}".format(run.info.lifecycle_stage))


# J'esperais qu'mlflow allait nous permettre de contourner le probleme de
# l'entrainement du modele, qui demande bcp d'espace memoire.
# probleme : mm sans l'ui, le tracking/logging mlflow consomment enormement !
# la solution a l'air pire que le probleme...


Name: knn_optimisation
Fitting 5 folds for each of 1 candidates, totalling 5 fits


: 

## Annexes, tests


### rf


In [ ]:
# test

from sklearn.preprocessing import MultiLabelBinarizer

def predict_tags_using_rf(train_df=train[::100], feature='title_nltk', target='all_tags', test_df=test, n_estimators=50):
    documents = train_df[feature].tolist()
    gensim_dictionary = Dictionary(documents)
    corpus = [gensim_dictionary.doc2bow(doc) for doc in documents]

    # Convert Gensim corpus to dense matrix
    dense_matrix = corpus2dense(corpus, num_terms=len(gensim_dictionary)).T

    # Convert multi-label tags into binary format
    mlb = MultiLabelBinarizer()
    y_encoded = mlb.fit_transform(train_df[target])

    # Fit Random Forest model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    rf_model.fit(dense_matrix, y_encoded)

    # Predictions
    predictions = []
    min_range = 3000
    max_range = 3005
    for i in range(min_range, max_range):
        query_document = test_df[feature][i]
        print(f'doc {i} : {query_document}')
        print(f'real tags : {test[target][i]}')
        query_bow = gensim_dictionary.doc2bow(query_document)
        query_vector = corpus2dense([query_bow], num_terms=len(gensim_dictionary)).T

        # Predict tags using Random Forest
        prediction_encoded = rf_model.predict(query_vector.reshape(1, -1))

        # Convert back to original tag format
        predicted_tags = mlb.inverse_transform(prediction_encoded.reshape(1, -1))

        predictions.append(predicted_tags)
        print(f'predicted : {predictions[-1]}', '\n')

    return predictions

# Call the function with your DataFrame and the desired text feature and target tags
predictions_rf = predict_tags_using_rf()


## Deep learning Models
